***Vorlesung 'Syntax natürlicher Sprachen'***

---
# Vorlesung 11: Statistisches Parsing

In [1]:
import nltk

---
# 1. Probabilistische CFGs: Abschätzung von Regelwahrscheinlichkeiten

In [2]:
from collections import defaultdict
from nltk.corpus import treebank

In [3]:
def find_relevant_constructions(lhs, only_with=None):
    lhs_nt = nltk.grammar.Nonterminal(lhs)
    should_filter = only_with is not None
    if should_filter:
        filter_by = list(map(nltk.grammar.Nonterminal, only_with))
        def passes_filter(tup):
            for f in filter_by:
                if f not in tup:
                    return False
            return True

    counter = defaultdict(int)
    ### zähle Produktionen in treebank mit lhs als linker Seite ###
    ### und einer rechten Seite, für die passes_filter True liefert ###
    for tree in treebank.parsed_sents():  # [(S (NP DET N) (VP V NP))]
        for prod in tree.productions():   # [(S, NP VP), (NP, DET N)]
            if prod.lhs() == lhs_nt:
                if not should_filter or passes_filter(prod.rhs()):
                    counter[prod] += 1

    return [ (k, counter[k]) for k in sorted(counter.keys(), key=counter.__getitem__) ]

---
## 1.1: Herunterladen von Ressourcen

#### Laden Sie sich zunächst die Ressource `corpora/treebank` über den NLTK Download-Manager herunter.

In [4]:
# nltk.download()

---
## 1.2: Von Daten zu Regelwahrscheinlichkeiten

#### Gegeben sei folgende kontextfreie Grammatik:

In [5]:
grammar = nltk.CFG.fromstring("""
S -> NP VP
VP -> V NP PP
VP -> V NP
NP -> DET N
NP -> NP PP
PP -> P NP

DET -> "the" | "a"
N -> "boy" | "woman" | "telescope"
V -> "saw"
P -> "with"
""")

#### Sie modelliert sehr einfache Sätze der Form `SBJ` *saw* `OBJ` mit optionaler Präpositionalphrase am Ende. Diese Präpositionalphrase kann entweder der näheren Bestimmung des Objekts oder der näheren Bestimmung der in der Verbalphrase ausgedrückten Handlung dienen.

In [6]:
sentence = "the boy saw a woman with a telescope"
parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)

                 S                                  
     ┌───────────┴────────┐                          
     │                    VP                        
     │       ┌───────┬────┴─────────┐                
     │       │       │              PP              
     │       │       │         ┌────┴───┐            
     NP      │       NP        │        NP          
 ┌───┴───┐   │   ┌───┴────┐    │    ┌───┴──────┐     
DET      N   V  DET       N    P   DET         N    
 │       │   │   │        │    │    │          │     
the     boy saw  a      woman with  a      telescope

                 S                                  
     ┌───────────┴────────┐                          
     │                    VP                        
     │       ┌────────────┴────┐                     
     │       │                 NP                   
     │       │       ┌─────────┴────┐                
     │       │       │              PP              
     │       │       │         ┌────┴

---
 
#### Im folgenden sollen aus der Penn Treebank Wahrscheinlichkeiten für die einzelnen Regeln extrahiert werden, um dieser Ambiguität Herr zu werden.

#### Hier ein Beispiel für geparste Sätze in der Penn Treebank:

In [7]:
for tree in nltk.corpus.treebank.parsed_sents('wsj_0001.mrg'):
    print(tree)

(S
  (NP-SBJ
    (NP (NNP Pierre) (NNP Vinken))
    (, ,)
    (ADJP (NP (CD 61) (NNS years)) (JJ old))
    (, ,))
  (VP
    (MD will)
    (VP
      (VB join)
      (NP (DT the) (NN board))
      (PP-CLR (IN as) (NP (DT a) (JJ nonexecutive) (NN director)))
      (NP-TMP (NNP Nov.) (CD 29))))
  (. .))
(S
  (NP-SBJ (NNP Mr.) (NNP Vinken))
  (VP
    (VBZ is)
    (NP-PRD
      (NP (NN chairman))
      (PP
        (IN of)
        (NP
          (NP (NNP Elsevier) (NNP N.V.))
          (, ,)
          (NP (DT the) (NNP Dutch) (VBG publishing) (NN group))))))
  (. .))


#### Nutzen Sie das im NLTK enthaltene Sample der Penn Treebank (nach Installation unter `nltk.corpus.treebank` zu finden) zunächst zur Identifikation der für eine Disambiguierung nützlichen (Teil-)bäume der Penn Treebank.

#### *Hinweis:* Sie können sich bei der Analyse auf die häufigsten Konstruktionen der Baumbank beschränken.

---
### 1.2.1: Regelwahrscheinlichkeiten für VP-Regeln abschätzen



---

#### Zählen Sie zunächst für die V+NP+PP-Konstruktion, wie oft sie in der Penn Treebank vorkommen und berechnen Sie die relativen Häufigkeiten als Approximation der Regelwahrscheinlichkeiten:

$$P(V, N\!P, P\!P \mid V\!P) = \dfrac{count(V\!P \rightarrow V\:N\!P\:P\!P)}{count(V\!P \rightarrow \setminus*)}$$

**hier (vgl. Grammatik):**

$$= \dfrac{count(V\!P \rightarrow V\:N\!P\:P\!P)}{count(V\!P \rightarrow V\:N\!P\:P\!P) + count(V\!P \rightarrow V\:N\!P)}$$

In [8]:
constructions = find_relevant_constructions('VP')
constructions[-20:]

[(VP -> VBZ NP-PRD, 163),
 (VP -> VBN NP PP, 170),
 (VP -> VBN NP PP-CLR, 178),
 (VP -> VBP NP, 185),
 (VP -> VBZ SBAR, 197),
 (VP -> VBZ S, 215),
 (VP -> VBD S, 223),
 (VP -> VP CC VP, 234),
 (VP -> VBN NP, 250),
 (VP -> VB VP, 258),
 (VP -> VBZ NP, 261),
 (VP -> VBP VP, 337),
 (VP -> VBD VP, 361),
 (VP -> VBG NP, 375),
 (VP -> VBD NP, 378),
 (VP -> VBZ VP, 459),
 (VP -> VBD SBAR, 631),
 (VP -> MD VP, 759),
 (VP -> VB NP, 805),
 (VP -> TO VP, 1257)]

In [9]:
vp_v_np_pp_frq = 178 + 170
vp_v_np_without_frq = 805 + 378 + 375 + 261 + 250 + 185 + 163

vp_with_pp = vp_v_np_pp_frq / (vp_v_np_pp_frq + vp_v_np_without_frq)
vp_without = vp_v_np_without_frq / (vp_v_np_pp_frq + vp_v_np_without_frq)

(vp_with_pp, vp_without)

(0.12585895117540688, 0.8741410488245931)

---
### 1.2.2: Regelwahrscheinlichkeiten für NP-Regeln abschätzen

#### Zählen Sie anschließend, wie oft die NP+PP-Konstruktion in der Penn Treebank vorkommt und berechnen Sie die relativen Häufigkeiten als Approximation der Regelwahrscheinlichkeiten. Das Vorgehen wird in folgender Formel veranschaulicht:

$$P(N\!P, P\!P \mid N\!P) = \dfrac{count(N\!P \rightarrow \:N\!P\:P\!P)}{count(N\!P \rightarrow \setminus*)}$$

**hier:**

$$= \dfrac{count(N\!P \rightarrow \:N\!P\:P\!P)}{count(N\!P \rightarrow \:N\!P\:P\!P) + count(N\!P \rightarrow DET\:N\!P)}$$



In [10]:
constructions = find_relevant_constructions('NP')
constructions[-20:]

[(NP -> PRP, 280),
 (NP -> NNP NNP NNP, 282),
 (NP -> NP CC NP, 289),
 (NP -> NN NNS, 304),
 (NP -> DT NN NN, 313),
 (NP -> CD, 327),
 (NP -> DT NNS, 358),
 (NP -> NP PP-LOC, 363),
 (NP -> QP -NONE-, 365),
 (NP -> JJ NN, 390),
 (NP -> NP SBAR, 409),
 (NP -> JJ NNS, 653),
 (NP -> NNP NNP, 734),
 (NP -> DT JJ NN, 740),
 (NP -> NNP, 837),
 (NP -> NNS, 996),
 (NP -> NN, 1110),
 (NP -> -NONE-, 1225),
 (NP -> DT NN, 2020),
 (NP -> NP PP, 2188)]

In [11]:
np_np_pp_frq = 2188 + 363
np_n_without_frq = 2020 + 358

np_with_pp = np_np_pp_frq / (np_np_pp_frq + np_n_without_frq)
np_without = np_n_without_frq / (np_np_pp_frq + np_n_without_frq)

(np_with_pp, np_without)

(0.5175491986204098, 0.4824508013795902)

---
### 1.2.3: Regelwahrscheinlichkeiten für DET-Regeln abschätzen


#### Zählen Sie auch für die `DET`-Erweiterungen nach *a* bzw. *the*, wie oft sie in der Penn Treebank (`DT`) vorkommen und berechnen Sie die relativen Häufigkeiten als Approximation der Regelwahrscheinlichkeiten:

$$P(the \mid DET) = \dfrac{count(DET \rightarrow the)}{count(V\!P \rightarrow \setminus*)}$$

**hier:**

$$= \dfrac{count(DET \rightarrow the)}{count(DET \rightarrow the) + count(DET \rightarrow a/an)}$$

In [12]:
constructions = find_relevant_constructions('DT')
constructions[-30:]

[(DT -> 'Those', 6),
 (DT -> 'Another', 6),
 (DT -> 'neither', 7),
 (DT -> 'Both', 9),
 (DT -> 'Each', 9),
 (DT -> 'No', 10),
 (DT -> 'half', 10),
 (DT -> 'All', 12),
 (DT -> 'An', 18),
 (DT -> 'every', 19),
 (DT -> 'These', 22),
 (DT -> 'Some', 22),
 (DT -> 'both', 34),
 (DT -> 'That', 37),
 (DT -> 'each', 37),
 (DT -> 'This', 40),
 (DT -> 'another', 42),
 (DT -> 'those', 55),
 (DT -> 'these', 55),
 (DT -> 'no', 76),
 (DT -> 'that', 77),
 (DT -> 'all', 86),
 (DT -> 'any', 103),
 (DT -> 'A', 105),
 (DT -> 'some', 122),
 (DT -> 'this', 184),
 (DT -> 'an', 316),
 (DT -> 'The', 713),
 (DT -> 'a', 1874),
 (DT -> 'the', 4038)]

In [13]:
det_the_frq = 4038 + 713
det_a_frq = 1874 + 316 + 105 + 18

det_the = det_the_frq / (det_the_frq + det_a_frq)
det_a = det_a_frq / (det_the_frq + det_a_frq)

(det_the, det_a)

(0.6725651189127972, 0.3274348810872027)

---
### 1.2.4: Erstellen einer PCFG

#### Die aus den Daten extrahierten relativen Häufigkeiten sollen nun zur Erstellung einer probabilistischen kontextfreien Grammatik (PCFG)  genutzt werden.

In [14]:
(vp_with_pp, vp_without, np_with_pp, np_without, det_the, det_a)

(0.12585895117540688,
 0.8741410488245931,
 0.5175491986204098,
 0.4824508013795902,
 0.6725651189127972,
 0.3274348810872027)

In [15]:
pcfg = """
S -> NP VP     [1.0]
VP -> V NP PP  [{}]
VP -> V NP     [{}]
NP -> DET N    [{}]
NP -> NP PP    [{}]
PP -> P NP     [1.0]

DET -> "the"     [{}]
DET -> "a"       [{}]
N -> "boy"       [0.4]
N -> "woman"     [0.4]
N -> "telescope" [0.2]
V -> "saw"       [1.0]
P -> "with"      [1.0]
""".format(
    vp_with_pp, vp_without, np_without,
    np_with_pp, det_the, det_a
)
grammar = nltk.PCFG.fromstring(pcfg)
print(grammar)

Grammar with 13 productions (start state = S)
    S -> NP VP [1.0]
    VP -> V NP PP [0.125859]
    VP -> V NP [0.874141]
    NP -> DET N [0.482451]
    NP -> NP PP [0.517549]
    PP -> P NP [1.0]
    DET -> 'the' [0.672565]
    DET -> 'a' [0.327435]
    N -> 'boy' [0.4]
    N -> 'woman' [0.4]
    N -> 'telescope' [0.2]
    V -> 'saw' [1.0]
    P -> 'with' [1.0]


---
### 1.2.5: Verwendung zur Disambiguierung

#### Testen Sie Ihre so erstellte Grammatik nun, indem Sie folgenden Satz parsen:

- *the boy saw a woman with a telescope*

In [16]:
parser = nltk.ViterbiParser(grammar)
for tree in parser.parse("the boy saw a woman with a telescope".split()):
    print(tree)
    tree.pretty_print(unicodelines=True)

(S
  (NP (DET the) (N boy))
  (VP
    (V saw)
    (NP
      (NP (DET a) (N woman))
      (PP (P with) (NP (DET a) (N telescope)))))) (p=0.000117227)
                 S                                  
     ┌───────────┴────────┐                          
     │                    VP                        
     │       ┌────────────┴────┐                     
     │       │                 NP                   
     │       │       ┌─────────┴────┐                
     │       │       │              PP              
     │       │       │         ┌────┴───┐            
     NP      │       NP        │        NP          
 ┌───┴───┐   │   ┌───┴────┐    │    ┌───┴──────┐     
DET      N   V  DET       N    P   DET         N    
 │       │   │   │        │    │    │          │     
the     boy saw  a      woman with  a      telescope



---

###  1.2.6 Wenn Sie sich die extrahierten Wahrscheinlichkeiten und das disambiguierte Ergebnis ansehen, überrascht Sie dann das Ergebnis der Syntaxanalyse?

In [17]:
# relevante Teilwahrscheinlichkeit VP-Attachment:
vp_with_pp * np_without
#P(VP->VP+NP+PP) * P(NP->DET+N)

0.060720751855369764

In [18]:
# relevante Teilwahrscheinlichkeit NP-Attachment (s.u.):
vp_without * np_with_pp * np_without
#P(VP->VP+NP) * P(NP->NP+PP) * P(NP->DET+N)

0.218266049165406

---

### 1.2.7 Vergleichen Sie dieses Ergebnis mit der PCFG-Analyse mit folgenden abweichenden Regelwahrscheinlichkeiten. 

### Warum wird hier, obwohl weiter `vp_with_pp < np_with_pp` gilt, der VP-Attachment-Baum als der wahrscheinlichere ausgewählt? 

##### (Beachten Sie die Anzahl an Regelanwendungen in den beiden Syntaxbäumen!)


In [19]:
vp_with_pp = 0.2
vp_without = 0.8
np_with_pp = 0.22
np_without = 0.78

In [20]:
pcfg = """
S -> NP VP     [1.0]
VP -> V NP PP  [{}]
VP -> V NP     [{}]
NP -> DET N    [{}]
NP -> NP PP    [{}]
PP -> P NP     [1.0]

DET -> "the"     [{}]
DET -> "a"       [{}]
N -> "boy"       [0.4]
N -> "woman"     [0.4]
N -> "telescope" [0.2]
V -> "saw"       [1.0]
P -> "with"      [1.0]
""".format(
    vp_with_pp, vp_without, np_without,
    np_with_pp, det_the, det_a
)

grammar = nltk.PCFG.fromstring(pcfg)
print(grammar)


Grammar with 13 productions (start state = S)
    S -> NP VP [1.0]
    VP -> V NP PP [0.2]
    VP -> V NP [0.8]
    NP -> DET N [0.78]
    NP -> NP PP [0.22]
    PP -> P NP [1.0]
    DET -> 'the' [0.672565]
    DET -> 'a' [0.327435]
    N -> 'boy' [0.4]
    N -> 'woman' [0.4]
    N -> 'telescope' [0.2]
    V -> 'saw' [1.0]
    P -> 'with' [1.0]


In [21]:
parser = nltk.ViterbiParser(grammar)
for tree in parser.parse("the boy saw a woman with a telescope".split()):
    print(tree)
    tree.pretty_print(unicodelines=True)

(S
  (NP (DET the) (N boy))
  (VP
    (V saw)
    (NP (DET a) (N woman))
    (PP (P with) (NP (DET a) (N telescope))))) (p=0.000219002)
                 S                                  
     ┌───────────┴────────┐                          
     │                    VP                        
     │       ┌───────┬────┴─────────┐                
     │       │       │              PP              
     │       │       │         ┌────┴───┐            
     NP      │       NP        │        NP          
 ┌───┴───┐   │   ┌───┴────┐    │    ┌───┴──────┐     
DET      N   V  DET       N    P   DET         N    
 │       │   │   │        │    │    │          │     
the     boy saw  a      woman with  a      telescope



In [22]:
# relevante Teilwahrscheinlichkeit VP-Attachment:
vp_with_pp * np_without
#P(VP->VP+NP+PP) * P(NP->DET+N)

0.15600000000000003

In [23]:
# relevante Teilwahrscheinlichkeit NP-Attachment (1 Regel mehr im Baum, wegen rekursiver Regel NP->NP+PP):
vp_without * np_with_pp * np_without
#P(VP->VP+NP) * P(NP->NP+PP) * P(NP->DET+N)

0.13728

---
#### vgl. die Anzahl der Regeln in den beiden Bäumen:

In [24]:
sentence = "the boy saw a woman with a telescope"
parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)
    for i, prod in enumerate(tree.productions()):
        print(i, prod)

                 S                                  
     ┌───────────┴────────┐                          
     │                    VP                        
     │       ┌───────┬────┴─────────┐                
     │       │       │              PP              
     │       │       │         ┌────┴───┐            
     NP      │       NP        │        NP          
 ┌───┴───┐   │   ┌───┴────┐    │    ┌───┴──────┐     
DET      N   V  DET       N    P   DET         N    
 │       │   │   │        │    │    │          │     
the     boy saw  a      woman with  a      telescope

0 S -> NP VP
1 NP -> DET N
2 DET -> 'the'
3 N -> 'boy'
4 VP -> V NP PP
5 V -> 'saw'
6 NP -> DET N
7 DET -> 'a'
8 N -> 'woman'
9 PP -> P NP
10 P -> 'with'
11 NP -> DET N
12 DET -> 'a'
13 N -> 'telescope'
                 S                                  
     ┌───────────┴────────┐                          
     │                    VP                        
     │       ┌────────────┴────┐                

---
# 2. Dependency Parsing mit übergangsbasiertem Shift-Reduce-Parser

In [25]:
from spacy import displacy

def transform_nr_conll(sent_nr):
    sent_list = []
    for line in list(filter(None, sent_nr.split("\n"))):
        line_list = line.split()
        line_list.pop(0)
        line_list.insert(1,"_")
        sent_list.append(" ".join([i for i in line_list[0:]]))

    return "\n".join([i for i in sent_list[0:]])



from nltk import DependencyGraph
from itertools import chain

def _tree_labeled(self, i):
        node = self.get_by_address(i)
        word = node["word"]
        rel = node["rel"]        
        deps = sorted(chain.from_iterable(node["deps"].values()))

        if deps:
            return Tree(word+'('+rel+')', [self._tree_labeled(dep) for dep in deps])
        else:
            return word+'('+rel+')'
        
def tree_labeled(self):
        node = self.root

        word = node["word"]
        rel = node["rel"]
        deps = sorted(chain.from_iterable(node["deps"].values()))
        return Tree(word+'('+rel+')', [self._tree_labeled(dep) for dep in deps])

DependencyGraph._tree_labeled = _tree_labeled
DependencyGraph.tree_labeled = tree_labeled



def displacy_dep_input(sent):
    deps = []
    for dep in sent.split('\n'):
        deps.append(dep.split())

    deps = [x for x in deps if x]

    ex = []
    word_list = []
    arc_list = []

    for index, dep in enumerate(deps):
        word_list.append({"text": dep[0], "tag": ""})
        line = index+1
        head = int(dep[2])
        label = dep[3]
        if head>line:
            start = index
            end = head-1
            direction = "left"
        else:
            start = head-1
            end = index  
            direction = "right"
        if(label.lower() != "root"):
            arc_list.append({"start": start, "end": end, "label": label, "dir": direction})

    ex.append({
        "words": word_list,
        "arcs": arc_list
    })    

    return ex

---
## 2.1 Einführungsbeispiel

### Ausgangspunkt (Trainingsdaten): Relationen zwischen Wörter

In [26]:
sent_nr = """
1 Book 0 ROOT
2 me 1 rel
3 the 5 rel
4 morning 5 rel
5 flight 1 rel
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

### Shift-Reduce-Parsing: 

- SHIFT: Wörter auf Stack schieben
- REDUCE: wenn Relation zwischen obersten Elementen auf Stack vorliegt:
    - ***Dependent von Stack löschen***
        - **ARC-Typ angeben**
        - **REDUCE-Reihenfolge nummerieren**

##### RIGHTARC-Einschränkung: RIGHTARC erst, wenn Dependent nicht mehr Kopf sein kann! (ggf. erst SHIFT)


In [27]:
sent_nr = """
1 Book 0 ROOT
2 me 1 RIGHTARC-1
3 the 5 LEFTARC-3
4 morning 5 LEFTARC-2
5 flight 1 RIGHTARC-4
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

---
### Erklärung ARC-Typ (LEFT/RIGHT):

`2 me 1 RIGHTARC-1`  (RIGHTARC: Dependent rechts von Head - Dependent-Index > Head-Index)

`3 the 5 LEFTARC-2` (LEFTARC: Dependent links von Head - Dependent-Index < Head-Index)

---
### Erklärung Nummerierung REDUCE-Reihenfolge:

##### siehe Stack-Zustände:

`>>> 2 x SHIFT:`

book me

`>>> REDUCE-1 (RIGHTARC, me loeschen):`

book

`>>> 3 x SHIFT (da keine Relation zwischen obersten Stackelementen):`

book the morning flight

`>>> REDUCE-2 (LEFTARC, morning loeschen):`

book the flight

`>>> REDUCE-3 (LEFTARC, the loeschen):`

book flight

`>>> REDUCE-4 (RIGHTARC, flight loeschen):`

book (=ROOT)


---
## 2.2 Beispiel mit RIGHTARC-Einschränkung


In [28]:
sent_nr = """
1 canceled 0 ROOT
2 flights 1 rel
3 to 4 rel
4 Houston 2 rel
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

##### RIGHTARC-Einschränkung: RIGHTARC erst, wenn Dependent nicht mehr Kopf sein kann! (ggf. erst SHIFT)

In [29]:
# 
sent_nr = """
1 canceled 0 ROOT
2 flights 1 RIGHTARC-3
3 to 4 LEFTARC-1
4 Houston 2 RIGHTARC-2
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

--- 
### Erklärung Nummerierung REDUCE-Reihenfolge:

##### siehe Stack-Zustände:

`>>> 2 x SHIFT:`

canceled flights

`>>> 2 x SHIFT (nicht REDUCE, da RIGHTARC-Einschränkung: flights Kopf von Dependenten, würde zu früh gelöscht):`

canceled flights to Houston

`>>> REDUCE-1 (LEFTARC, to loeschen):`

canceled flights Houston

`>>> REDUCE-2 (RIGHTARC, Houston loeschen):`

canceled flights

`>>> REDUCE-3 (RIGHTARC, flights loeschen):`

canceled (=ROOT)


---
## 2.3 Dependency-Parsing nicht-projektiver Strukturen

- nicht projektive Strukturen = *long distance dependency*, diskontinuierliche Strukturen
    - vgl. https://en.wikipedia.org/wiki/Discontinuity_(linguistics)#Projectivity


- können mit Dependenzgrammatiken modelliert werden (abstrahiert von linearer Anordnung) 
- aber: **nur bestimmte Dependency Parsingalgorithmen erlauben die Verarbeitung solcher Strukturen**


### 2.3.1 Einführungsbeispiel:

In [30]:
# 
sent_nr = """
1 x 2 rel
2 y 0 ROOT
3 z 1 rel
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

##### kein Pfand von x nach y! (nicht-projektiv)

#### Problem beim SHIFT-REDUCE-PARSING (kein vollständiger Parse):
```
x y 
>LEFTARC
y
> SHIFT
y z
```


#### auch mit zusätzlicher LEFTARC-Einschränkung nicht verarbeitbar:
```
x y 
> SHIFT statt LEFTARC:
x y z
```

---
### 2.3.2 Beispiel für eine nicht-projektive Struktur: rechtsversetzter Relativsatz (adnominaler Clause)

- ins Nachfeld extrahierter Relativsatz

In [31]:
# 
sent_nr = """
1 Jeder 2 nsubj
2 jubelt 0 ROOT
3 der 4 nsubj
4 gewinnt 1 acl:relcl
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

### vgl. NLTK-Dependency-Parser: Verarbeitung nicht-projektiver Strukturen nur mit entsprechendem Parser:


In [32]:
grammar = nltk.DependencyGrammar.fromstring("""
    'jubelt' -> 'Jeder' 
    'Jeder' -> 'gewinnt' 
    'gewinnt' -> 'der' 
    """)

#### projektiver Satz wird mit `ProjectiveDependencyParser` geparst:

In [33]:
sent = 'Jeder der gewinnt jubelt'.split()
print(sent)

['Jeder', 'der', 'gewinnt', 'jubelt']


In [34]:
parser = nltk.ProjectiveDependencyParser(grammar)

for tree in parser.parse(sent):
    print(tree, "\n")
    tree.pretty_print(unicodelines=True)

(jubelt (Jeder (gewinnt der))) 

 jubelt
   │    
 Jeder 
   │    
gewinnt
   │    
  der  



#### nicht-projektiver Satz ist nur mit  `NonprojectiveDependencyParser` verarbeitbar:

In [35]:
sent = 'Jeder jubelt der gewinnt'.split()
print(sent)

['Jeder', 'jubelt', 'der', 'gewinnt']


In [36]:
#kein Parse mit ProjectiveDependencyParser!
parser = nltk.ProjectiveDependencyParser(grammar)

for tree in parser.parse(sent):
    print(tree, "\n")
    tree.pretty_print(unicodelines=True)

In [37]:
#NonprojectiveDependencyParser: erkennt Struktur korrekt
#Code s. http://www.nltk.org/howto/dependency.html

dp = nltk.NonprojectiveDependencyParser(grammar)
g, = dp.parse(sent)

print(g.root['word'])

for _, node in sorted(g.nodes.items()):
    if node['word'] is not None:
        print('{address} {word}: {d}'.format(d=node['deps'][''], **node))

print('\n', g.tree(), '\n')
g.tree().pretty_print(unicodelines=True)

jubelt
1 Jeder: [4]
2 jubelt: [1]
3 der: []
4 gewinnt: [3]

 (jubelt (Jeder (gewinnt der))) 

 jubelt
   │    
 Jeder 
   │    
gewinnt
   │    
  der  

